<hr style="height:2px;border:none"/>
<img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">

<center>
    <h1>INF495 - Modelamiento Computacional Aplicado</h1>
    <h1>Desafío 3</h1>
    <h3>José Quezada</h3>
</center>
<br>
<hr style="height:2px;border:none"/>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# NOTA - - - - 
# radio nacional: 232326169 (promocion de mente vital: $23k)
#  - - - - - -


In [3]:
class bicicleta:
    def __init__(self, vMax, tReaccion, initAcc, dist):
        self.loc = [0,0]
        self.vAct = int(vMax/2) #probando
        self.vMax = vMax
        self.tReaccion = tReaccion
        self.initAcc = initAcc
        self.actAcc = initAcc
        
        self.dist = dist


class pista:
    def __init__(self, largo, ancho):
        self.largo = largo
        self.map = np.zeros((ancho, largo))

In [4]:
# elimina un elemento de un arreglo de manera eficiente
# además elimina los datos del elemento del mapa
def swapop(Poblacion, index, mapa):

    # print("pob[swapop]:", [p.loc for p in Poblacion])
   
    coordMapaDead = Poblacion[index].loc
    coordMapaLast = Poblacion[-1].loc

    # si el elemento a ser eliminado no es el último
    if index < len(Poblacion) and index != -1 and index >= 0:

        # guardar en aux datos del ciclista finalizado
        aux = Poblacion[index]
        Poblacion[index] = Poblacion[-1]
        Poblacion[-1] = aux
    
    elif index < 0:
        print("\n[swapop]: error: index menor a 0")

    # entregarle posición correcta al ciclista desplazado
    Poblacion[index].loc = coordMapaLast
    mapa[coordMapaLast[0], coordMapaLast[1]] = 1
    
    # eliminar datos del mapa
    # mapa = liberarCoord(mapa, coordMapaDead)
    mapa[coordMapaDead[0], coordMapaDead[1]] = 0

    # print("coordMapaLast:", coordMapaLast)
    # print("coordMapaDead:", coordMapaDead)

    # eliminar último elemento de arreglo
    Poblacion.pop()

    # print("pob[swapop] fin:", [p.loc for p in Poblacion])

    return Poblacion, mapa


# retorna 'a': accidente, 's': si, 'n': no
def bicicleta_cercana(Pista, Poblacion, pIndex):

    # posicion
    loc = Poblacion[pIndex].loc

    # limite del radio
    lim = loc[1] + radioCerc
    if lim >= Pista.largo:
        lim = Pista.largo - 1

    index = 1
    for i in Pista.map[loc[0], loc[1] + 1 : lim + 1]:
        if i == 1:
            if index < 3:
                
                if debug:
                    print("accidente!")

                Pista.map[loc[0], lim] = 0
                Pista.map[loc[0], loc[1]] = 0

                return Pista, index

            else:
                # solo hay que frenar
                return Pista, 1111 

        index += 1

    # no hay nadie cerca
    return Pista, 9999 


# encontrar bicicletas por su ubicación
def findByLoc(Poblacion, loc):

    pFound = []
    index = 0

    for p in Poblacion:
        if p.loc == loc:
            pFound.append(index)
        index += 1

    return pFound


In [5]:
def plotData(pobs, meta, accident):

    pobs_count = len(pobs) - 1
    x = np.linspace(0, pobs_count, pobs_count + 1)
    
    plt.figure()
    plt.title("Flujos de ciclistas")
    plt.plot(x, meta, 'b.', label='Meta')
    plt.plot(x, accident, 'r.', label='Accidentados')
    plt.legend(loc='best')
    plt.xlabel('Población')
    # plt.ylabel('Estados')
    plt.grid(True)
    plt.show()

In [6]:
def main():

    # inicializar pista
    largoP = 1000 # va a depender de la granularidad y distancia
    P = pista(largoP, 2)

    t = 0
    T = 100

    # población de bicicletas
    Pob = []

    cantTotal = 0
    cantMeta = 0
    cantAccident = 0

    while t < T:


        if t % flujo_in == 0 and t < 40:
            vInit = np.random.choice([10,20,30], p= [0.3,0.6,0.1])
            # tReaccion depende de la velocidad máxima y tiempo de frenado
            tReaccion = 1
            # que en 4 pasos alcance la velocidad máxima
            accInit = int(vInit/4)

            vis = np.random.choice(np.linspace(0,1,10))

            Pob.append(bicicleta(vInit, tReaccion, accInit, vis))
            cantTotal += 1

            P.map[Pob[-1].loc[0], Pob[-1].loc[1]] = 1

        pobFin = []
        b_index = 0
        for _ in Pob:

            P, bc = bicicleta_cercana(P, Pob, b_index)
            
            # Accidente
            if bc <= radioCerc:
                
                # buscar en Pob por loc
                pobFin.append(b_index)
                cantAccident += 1

                if debug:
                    print("accidentados:\n", b_index)

                # bicicletas afectadas
                accidentados = findByLoc(Pob, [Pob[b_index].loc[0], Pob[b_index].loc[1] + bc])
                for ac in accidentados:
                    pobFin.append(ac)
                    cantAccident += 1
                    if debug:
                        print(ac)

                b_index += 1
                continue

            # Frenar
            elif bc == 1111:
                Pob[b_index].actAcc = accFreno

            # Dejar de frenar
            elif bc == 9999 and Pob[b_index].actAcc == accFreno:
                Pob[b_index].actAcc = Pob[b_index].initAcc

            # aceleración de frenado variable??

            # si pasa un tiempo con aceleracion 0 y hay una bici a una distancia un poquito mayor a una cercana
            # tiene que pasar a la bici, cambiando de pista

            # actualizar velocidad con aceleración actual (independiente de la situación)
            prev_loc = Pob[b_index].loc
            if Pob[b_index].vAct < Pob[b_index].vMax:
                Pob[b_index].vAct += Pob[b_index].actAcc

            new_loc = Pob[b_index].loc[1] + Pob[b_index].vAct

            # Meta
            if new_loc >= P.largo:
                pobFin.append(b_index)
                cantMeta += 1
                b_index += 1
                continue
            else:
                Pob[b_index].loc = [Pob[b_index].loc[0], new_loc]


            # print(Pob[b_index].vAct)
            # actualizar mapa
            P.map[Pob[b_index].loc[0], int(Pob[b_index].loc[1])] = 1
            P.map[prev_loc[0], int(prev_loc[1])] = 0

            # print(b_index, ":", prev_loc, Pob[b_index].loc)
            b_index += 1


        # Eliminar bicicletas de Poblacion
        pobFin.sort()
        if debug and len(pobFin) > 0:
            print(t, pobFin)
        for pf in reversed(pobFin):
            Pob, P.map = swapop(Pob, pf, P.map)
        
        if debugMap:
            Y, X = np.where(P.map)

            fig, ax = plt.subplots(figsize=(12, 4))
            ax.plot(X,Y, ".")
            plt.xlim([0, P.largo])

            plt.show()
            plt.pause(0.01)
            clear_output(wait=True)

        t += 1

    # graficar datos
    print("Total:", cantTotal, "\nMeta:", cantMeta, "\nAccidentados:", cantAccident)

    # plotData(pobs, meta, accident)

In [7]:
np.random.seed(8)

flujo_in = 4

accFreno = -10
radioCerc = 30

debug = False
debugMap = True 

main()

Total: 10 
Meta: 3 
Accidentados: 6
